In [ ]:
# 사용방법 - 복사하여 주석만 제거하여 사용
import sys
from pathlib import Path

# 현재 노트북(.ipynb)이 src/ 안에 있으니 상위 폴더(프로젝트 루트)를 추가
BASE_DIR    = Path().resolve().parent.parent    # Notebook이 src/ 안이라면 .parent
config_path = BASE_DIR / 'config' / 'data_paths.yaml'
sys.path.insert(0, str(BASE_DIR))


# 이제 바로 import
from scripts.data_loader import load_data

# 테스트
df_pm25 = load_data('pm25')
df_pm10 = load_data('pm10')
df_a = load_data('asthma')
#df_r = load_data('rhinitis')

# or 
# df = load_data('pm10', section='raw')

# # 1. "미세먼지 농도가 높으면 질병율이 높을 것이다."
귀무가설 H₀: 미세먼지 농도와 질병율 사이에 양(+)의 상관이 없다 (ρ ≤ 0)

대립가설 H₁: 미세먼지 농도가 높을수록 질병율이 높다 (ρ > 0)

# 1. "미세먼지 농도가 높으면 질병율이 높을 것이다."
귀무가설 H₀: 미세먼지 농도와 질병율 사이에 양(+)의 상관이 없다 (ρ ≤ 0)

대립가설 H₁: 미세먼지 농도가 높을수록 질병율이 높다 (ρ > 0)

In [ ]:
x = df['avg_pm10']
y = df['episode_count']

# 상관계수 계산 (피어슨)
corr, p_value = pearsonr(x, y)

# 단측 검정 (ρ > 0)
p_one_tailed = p_value / 2 if corr > 0 else 1 - p_value / 2

print(f"상관계수 (r): {corr:.4f}")
print(f"단측 검정 p-value: {p_one_tailed:.4f}")

alpha = 0.05
if p_one_tailed < alpha:
    print("대립가설 채택: 미세먼지 농도가 높을수록 질병율도 높다.")
else:
    print("귀무가설 채택: 미세먼지 농도와 질병율 간의 양의 상관은 유의하지 않다.")

상관계수 (r): 0.5883
단측 검정 p-value: 0.0000
대립가설 채택: 미세먼지 농도가 높을수록 질병율도 높다.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl  
from scipy import stats
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
df_pm25

,구분,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,수원,...,함양,남해,산청,의령,창녕,합천,도평균.7,제주,서귀포,도평균.8
0,2015.01 월,25,27,33,31,27,34,27,NaN,29,...,NaN,NaN,NaN,NaN,NaN,NaN,26,24,NaN,NaN
1,2015.02 월,30,31,33,33,33,36,32,NaN,34,...,NaN,NaN,NaN,NaN,NaN,NaN,31,25,NaN,NaN
2,2015.03 월,30,27,29,31,35,37,29,NaN,33,...,NaN,NaN,NaN,NaN,NaN,NaN,29,31,NaN,NaN
3,2015.04 월,21,24,22,30,24,23,25,NaN,23,...,NaN,NaN,NaN,NaN,NaN,NaN,23,23,NaN,NaN
4,2015.05 월,22,31,25,28,24,29,31,NaN,24,...,NaN,NaN,NaN,NaN,NaN,NaN,28,22,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2024.06 월,18,16,15,18,12,15,20,15.0,18,...,15.0,12.0,15.0,15.0,17.0,15.0,15,12,11.0,12.0
114,2024.07 월,12,8,8,13,6,8,12,7.0,12,...,6.0,7.0,9.0,9.0,11.0,9.0,8,7,6.0,7.0
115,2024.08 월,16,11,10,18,12,12,13,12.0,15,...,14.0,13.0,13.0,14.0,15.0,14.0,13,13,11.0,12.0
116,2024.09 월,9,8,8,11,10,8,10,8.0,10,...,11.0,8.0,10.0,11.0,11.0,10.0,10,10,10.0,10.0


In [ ]:
target_city = '서울특별시'
#df_seoul.to_excel(f'C:/Users/shinp/Downloads/비염/{target_city} 비염중간간.xlsx', index=False)
# # '요양개시연월' 기준으로 진료에피소드 건수 합산
df_a_group = df_a[df_a['시도명']== target_city].groupby('요양개시연월')['진료에피소드 건수'].sum().reset_index()
#df_a_group = df_a[df_a['시군구명']== target_city].groupby('요양개시연월')['진료에피소드 건수'].sum().reset_index()

df_pm25_group = df_pm25[['구분', target_city]]
df_pm10_group = df_pm10[['구분', target_city]]
print(df_a_group)
print(df_pm25_group)
print(df_pm10_group)

      요양개시연월  진료에피소드 건수
0    2006-01     105163
1    2006-02      53052
2    2006-03      65642
3    2006-04      70603
4    2006-05      60590
..       ...        ...
211  2023-08      25590
212  2023-09      29118
213  2023-10      35982
214  2023-11      36405
215  2023-12      33634

[216 rows x 2 columns]
            구분  서울특별시
0    2006.01 월     65
1    2006.02 월     67
2    2006.03 월     81
3    2006.04 월    107
4    2006.05 월     63
..         ...    ...
221  2024.06 월     31
222  2024.07 월     20
223  2024.08 월     24
224  2024.09 월     16
225  2024.10 월     23

[226 rows x 2 columns]


In [ ]:
# 1) df_pm의 '구분'을 'YYYY-MM' 문자열로 바꾸기
df_pm10_group['month'] = (
    df_pm10_group['구분']
      .str.replace(' 월', '', regex=False)     # "2006.01 월" → "2006.01"
      .str.replace('.', '-', regex=False)      # "2006.01"    → "2006-01"
)
df_pm25_group['month'] = (
    df_pm10_group['구분']
      .str.replace(' 월', '', regex=False)     # "2006.01 월" → "2006.01"
      .str.replace('.', '-', regex=False)      # "2006.01"    → "2006-01"
)

# 날짜형으로 변환해두면 나중에 편합니다
df_pm25_group['month'] = pd.to_datetime(df_pm25_group['month'], format='%Y-%m')
df_pm10_group['month'] = pd.to_datetime(df_pm10_group['month'], format='%Y-%m')
df_a_group['month'] = pd.to_datetime(df_a_group['요양개시연월'], format='%Y-%m')

# 공통된 'month' 컬럼으로 두 df 합치기
df_merged1 = pd.merge(
    df_pm10_group,
    df_a_group[['month', '진료에피소드 건수']],
    on='month',
    how='inner'   # 공통된 월만, 빠진 달도 모두 보고 싶으면 how='outer' 사용
)
df_merged1 = df_merged1.rename(columns={
     target_city : 'avg_pm10',
     '진료에피소드 건수': 'episode_count'
 })
df_merged1 = df_merged1.drop(columns=['구분'])

df_merged2 = pd.merge(
    df_merged1,
    df_pm25_group,
    on='month',
    how='inner'   # 공통된 월만, 빠진 달도 모두 보고 싶으면 how='outer' 사용
)
df_merged2 = df_merged2.rename(columns={
     target_city : 'avg_pm25',
 })
df_merged2 = df_merged2.drop(columns=['구분'])

# # 컬럼 순서 재배치: month를 맨 앞으로
df = df_merged2[['month', 'avg_pm10', 'avg_pm25','episode_count']]

# # 확인
print(df.head())

       month  avg_pm10  avg_pm25  episode_count
0 2006-01-01        65        25         105163
1 2006-02-01        67        30          53052
2 2006-03-01        81        30          65642
3 2006-04-01       107        21          70603
4 2006-05-01        63        22          60590


C:\Users\shinp\AppData\Local\Temp\ipykernel_34340\2874563643.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pm10_group['month'] = (
C:\Users\shinp\AppData\Local\Temp\ipykernel_34340\2874563643.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pm25_group['month'] = (
C:\Users\shinp\AppData\Local\Temp\ipykernel_34340\2874563643.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

# 회귀분석

In [ ]:
X = df[["avg_pm10", "avg_pm25"]]
y = df["episode_count"]

# 80% train / 20% test 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"📊 Mean Squared Error (MSE): {mse:.2f}")
print(f"📈 R-squared: {r2:.3f}")

# 예측값 vs 실제값 시각화
plt.scatter(y_test, y_pred)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')  # 대각선
plt.xlabel("Actual Episode Count")
plt.ylabel("Predicted Episode Count")
plt.title("Actual vs Predicted (Test Set)")
plt.grid(True)
plt.show()

In [ ]:
# 사용방법 - 복사하여 주석만 제거하여 사용
import sys
from pathlib import Path

# 현재 노트북(.ipynb)이 src/ 안에 있으니 상위 폴더(프로젝트 루트)를 추가
BASE_DIR    = Path().resolve().parent.parent    # Notebook이 src/ 안이라면 .parent
config_path = BASE_DIR / 'config' / 'data_paths.yaml'
sys.path.insert(0, str(BASE_DIR))


# 이제 바로 import
from scripts.data_loader import load_data

🎯 귀무가설(H₀):
성별에 따라 미세먼지의 질병 유발력은 차이가 없다.
→ pm10:gender 상호작용 항의 회귀계수 = 0

🆚 대립가설(H₁):
성별에 따라 미세먼지의 질병 유발력이 다르다.
→ pm10:gender 상호작용 항의 회귀계수 ≠ 0